In [1]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from dotenv import load_dotenv
from langchain.llms import OpenAI
import os

#### OpenAI LLM & API key setup

In [2]:
 dotenv_path = "C:/Users/burha/OneDrive/Desktop/Career/api_key.env"
load_dotenv(dotenv_path)

True

In [3]:
api_key = os.getenv("api_key")
llm = OpenAI(openai_api_key=api_key, temperature=0.2)

#### Connect with database

In [17]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "classicmodels"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmitt	Carine 	40.32.2555	54, rue Royale	None	Nantes	None	44000	France	1370	21000.00
112	Signal Gift Stores	King	Je

#### Few shot learning

In [30]:
few_shots = [
    {'Question' : "Who reports to William Patterson?",
     'SQLQuery' : "SELECT CONCAT(reports.firstname, ' ', reports.lastname) AS Employee FROM Employees boss JOIN Employees reports ON boss.employeeNumber = reports.reportsTo WHERE boss.firstName = 'William' and boss.lastName = 'Patterson';",
     'SQLResult': "Result of the SQL query",
     'Answer' : "'Andy Fixter', 'Peter Marsh', 'Tom King'"},
    {'Question': "Compute the commission for each sales representative, assuming the commission is 5% of the value of an order. Sort by employee last name and first name",
     'SQLQuery': "SELECT CONCAT(firstName, ' ',lastName) AS Name, FORMAT(.05 * SUM(quantityOrdered * priceEach),0) AS Commission FROM Employees JOIN Customers ON Employees.employeeNumber = Customers.salesRepEmployeeNumber JOIN Orders ON Customers.customerNumber = Orders.customerNumber JOIN OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber GROUP BY employeeNumber ORDER BY lastName, firstName ASC;",
     'SQLResult': "Result of the SQL query",
     'Answer': "'Loui Bondur, 28,474', 'Larry Bott, 36,605', 'Pamela Castillo, 43,411', 'Julie Firrelli, 19,333','Andy Fixter, 28,129', 'Martin Gerard, 19,374'......"},
    {'Question': "What are the top 3 products by quantity of on hand for products listed on 'On Hold' orders?" ,
     'SQLQuery' : "SELECT productName, FORMAT(quantityInStock,0) AS `Quantity in stock` FROM OrderDetails JOIN Orders ON Orders.orderNumber = OrderDetails.orderNumber JOIN Products on OrderDetails.productCode = Products.productCode WHERE status = 'On Hold' order by quantityInStock desc limit 3;",
     'SQLResult': "Result of the SQL query",
     'Answer': "'America West Airlines B757-200, 9,653', '2002 Chevy Corvette, 9,446', '1912 Ford Model T Delivery Wagon, 9,173'"} ,
     {'Question' : "What is the value of orders shipped in August 2004?" ,
      'SQLQuery': "SELECT FORMAT(SUM(quantityOrdered*priceEach),0) as orderValue FROM Orders JOIN OrderDetails ON Orders.orderNumber = OrderDetails. orderNumber AND YEAR(orderDate) = 2004 AND MONTH(orderDate) = 8;",
      'SQLResult': "Result of the SQL query",
      'Answer' : '419,327'},
    {'Question': "What is the difference in the amount received for each month of 2004 compared to 2003?",
     'SQLQuery' : "WITH t2003 AS (SELECT YEAR(paymentDate) AS 'year', MONTH(paymentDate) AS 'month', sum(amount) AS amount FROM Payments WHERE YEAR(paymentDate) = 2003 GROUP BY YEAR(paymentDate), MONTH(paymentDate)), t2004 AS (SELECT YEAR(paymentDate) AS 'year', MONTH(paymentDate) AS 'month', sum(amount) AS amount FROM Payments WHERE YEAR(paymentDate) = 2004 GROUP BY YEAR(paymentDate), MONTH(paymentDate)) SELECT t2003.month,format((t2004.amount - t2003.amount),2) AS variance FROM t2003 JOIN t2004 ON t2003.month = t2004.month ORDER BY t2003.month;",
     'SQLResult': "Result of the SQL query",
     'Answer' : "'1, 207,884.51', '2, -37,732.35', '3, 204,898.73'"}
]

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [28]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [31]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

In [32]:
to_vectorize

["Who reports to William Patterson? SELECT CONCAT(reports.firstname, ' ', reports.lastname) AS Employee FROM Employees boss JOIN Employees reports ON boss.employeeNumber = reports.reportsTo WHERE boss.firstName = 'William' and boss.lastName = 'Patterson'; Result of the SQL query 'Andy Fixter', 'Peter Marsh', 'Tom King'",
 "Compute the commission for each sales representative, assuming the commission is 5% of the value of an order. Sort by employee last name and first name SELECT CONCAT(firstName, ' ',lastName) AS Name, FORMAT(.05 * SUM(quantityOrdered * priceEach),0) AS Commission FROM Employees JOIN Customers ON Employees.employeeNumber = Customers.salesRepEmployeeNumber JOIN Orders ON Customers.customerNumber = Orders.customerNumber JOIN OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber GROUP BY employeeNumber ORDER BY lastName, firstName ASC; Result of the SQL query 'Loui Bondur, 28,474', 'Larry Bott, 36,605', 'Pamela Castillo, 43,411', 'Julie Firrelli, 19,333','Andy Fix

In [33]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [34]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "Who reports to Mary Patterson?"})

[{'Answer': "'Andy Fixter', 'Peter Marsh', 'Tom King'",
  'Question': 'Who reports to William Patterson?',
  'SQLQuery': "SELECT CONCAT(reports.firstname, ' ', reports.lastname) AS Employee FROM Employees boss JOIN Employees reports ON boss.employeeNumber = reports.reportsTo WHERE boss.firstName = 'William' and boss.lastName = 'Patterson';",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': "'Loui Bondur, 28,474', 'Larry Bott, 36,605', 'Pamela Castillo, 43,411', 'Julie Firrelli, 19,333','Andy Fixter, 28,129', 'Martin Gerard, 19,374'......",
  'Question': 'Compute the commission for each sales representative, assuming the commission is 5% of the value of an order. Sort by employee last name and first name',
  'SQLQuery': "SELECT CONCAT(firstName, ' ',lastName) AS Name, FORMAT(.05 * SUM(quantityOrdered * priceEach),0) AS Commission FROM Employees JOIN Customers ON Employees.employeeNumber = Customers.salesRepEmployeeNumber JOIN Orders ON Customers.customerNumber = Orders.customerNum

In [35]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [36]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### Setting up PromptTemplete using input variables

In [37]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [38]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [39]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [40]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [41]:
new_chain("Compute the profit generated by each sales representative based on the orders from the customers they serve. Sort by profit generated descending.")



> Entering new SQLDatabaseChain chain...
Compute the profit generated by each sales representative based on the orders from the customers they serve. Sort by profit generated descending.
SQLQuery:SELECT CONCAT(firstName, ' ',lastName) AS Name, FORMAT(SUM(quantityOrdered * priceEach - (quantityOrdered * buyPrice)),0) AS Profit FROM Employees JOIN Customers ON Employees.employeeNumber = Customers.salesRepEmployeeNumber JOIN Orders ON Customers.customerNumber = Orders.customerNumber JOIN OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber JOIN Products ON OrderDetails.productCode = Products.productCode GROUP BY employeeNumber ORDER BY Profit DESC;
SQLResult: [('Gerard Hernandez', '504,645'), ('Leslie Jennings', '435,208'), ('Pamela Castillo', '340,728'), ('Larry Bott', '290,204'), ('Barry Jones', '276,659'), ('George Vanauf', '269,596'), ('Loui Bondur', '234,891'), ('Peter Marsh', '230,812'), ('Andy Fixter', '222,207'), ('Steve Patterson', '197,879'), ('Foon Yue Tseng', '194,8

{'query': 'Compute the profit generated by each sales representative based on the orders from the customers they serve. Sort by profit generated descending.',
 'result': "'Gerard Hernandez, 504,645', 'Leslie Jennings, 435,208', 'Pamela Castillo, 340,728', 'Larry Bott, 290,204', 'Barry Jones, 276,659'"}